In [6]:
import pandas as pd

import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

import os
import time

In [5]:
df = '/content/drive/MyDrive/Colab Notebooks/코로나관련/대중교통관련/서울시버스정류소좌표데이터(2021.01.14.).csv'
df = pd.read_csv(df, encoding='euc-kr')
df.head()

,표준ID,ARS-ID,정류소명,X좌표,Y좌표
0,100000001,1001,종로2가사거리,126.987786,37.569764
1,100000002,1002,창경궁.서울대학교병원,126.996520,37.579179
2,100000003,1003,명륜3가.성대입구,126.998290,37.582709
3,100000004,1004,종로2가.삼일교,126.987507,37.568582
4,100000005,1005,혜화동로터리.여운형활동터,127.001694,37.586230


In [ ]:
bus_all = list()
for sub_stop in tqdm(sub_stops):
    query = sub_stop.replace(' ', "%20") + "역"
    sub_stop_name = ' '.join(sub_stop.split()[1:]) + "역"
    map_url = f'https://m.map.kakao.com/actions/searchView?q={query}'
    print(map_url, end=" : ")

    try:
        # 검색 결과 중 지하철역 이름이 같은 경우 찾음
        req = requests.get(map_url)
        soup = BeautifulSoup(req.text, 'html.parser')
        li_list = soup.find_all("li")
        print(f"{len(li_list)}개 검색결과")
        for li_ in li_list:
            if sub_stop_name in str(li_.find('span', {"class": "txt_tit"})):
                if "서울" in str(li_.find('span', {"class": "txt_g"})):
                    break
        
        # 해당 버스 정류장에서 다시 검색 -> 주소 크롤링
        sub_place = li_.find('span', {"class": "txt_g"}).text

        # 버스 정보 저장
        sub_all.append({"역명": sub_stop, "역_주소": sub_place})
        print(sub_stop, '/', sub_place)
        print()

    except Exception as e:
        print(e)
        sub_all.append({"역명": sub_stop, "역_주소": "NOT_FOUND"})
        print(sub_stop, '/ NOT_FOUND')
        print()
